In [14]:
import os
import config
import langchain.llms
from typing import Any, List, Optional, Dict
import google.generativeai as genai
from langchain.schema.runnable import Runnable
from langchain_core.language_models import LLM
from langchain.chains.base import Chain
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from pydantic import Field, PrivateAttr
from langchain.schema import BaseOutputParser

In [2]:
api_key = os.environ.get('API_KEY')
genai.configure(api_key=api_key)

### **Setting up the LLM Wrapper**

In [3]:
class GeminiLLM(LLM):
    """Wrapper around Google's Gemini model to make it compatible with LangChain."""
    
    model_name: str = Field(default="gemini-1.5-flash", description="The name of the Gemini model to use")
    _model: Any = PrivateAttr()
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._model = genai.GenerativeModel(model_name=self.model_name)
    
    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "gemini"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Execute the LLM with the given prompt."""
        response = self._model.generate_content(prompt)
        return response.text

# Create and use the wrapper
gemini_llm = GeminiLLM()

In [12]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash')

class GeminiRunnable(Runnable):
    def __init__(self, model):
        self.model = model

    def invoke(self, prompt, *args, **kwargs):
        return  self.model.generate_content(prompt)  

# Wrap Gemini model in a Runnable
gemini_runnable = GeminiRunnable(model)

response = gemini_runnable.invoke("in one sentence, what is LLM.")
print(response.text)

LLMs are large language models, AI systems trained on massive datasets to understand and generate human-like text.



## creating an SQL database object

In [4]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "root123"
db_host = "localhost"
db_name = "atliq_tshirts"

try:
    # Attempt to create a database connection
    db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
    print("DATABASE CONNECTED SUCCESSFULLY")
except Exception as e:
    # Print failure message and the specific error
    print("DATABASE CONNECTION FAILED")
    print(f"Error: {str(e)}")
    exit(1)

DATABASE CONNECTED SUCCESSFULLY


In [5]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

### **creating an SQL Database chain**

In [33]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(gemini_llm, db, return_direct=False)
response = db_chain.run("How many t-shirts do we have left for nike in extra small and white color?")

In [34]:
print(response)

Answer: There are no t-shirts from Nike that are extra small and white.  The query returned an empty set.


In [35]:
response


'Answer: There are no t-shirts from Nike that are extra small and white.  The query returned an empty set.'